In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [3]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [4]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [5]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [6]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error i

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6849 - loss: 0.5986  
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5365 - loss: 0.7030
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7676 - loss: 0.5095
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7469 - loss: 0.5532
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7635 - loss: 0.5156
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7609 - loss: 0.5125
  1/167 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9062 - loss: 0.3503Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6442 - loss: 0.6219
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7712 - loss: 0.5131
Epoch 2/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8143 - loss: 0.4296  
Epoch 3/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.80

/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 993us/step - accuracy: 0.8545 - loss: 0.3298
Epoch 50/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8513 - loss: 0.3400  
117/167 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - accuracy: 0.8666 - loss: 0.3242Epoch 48/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8601 - loss: 0.3304
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 961us/step - accuracy: 0.8632 - loss: 0.3371
Epoch 48/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8658 - loss: 0.3308
Epoch 48/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8724 - loss: 0.3148
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - accuracy: 0.8664 - loss: 0.3241
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step - accuracy: 0.8574 - loss: 0.3322
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 812us/step - accuracy: 0.8605 - loss: 0.3421
Epoch 49/50
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.8637 - loss: 0.3106
Epoch 50/50
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 664us/step
167/167 ━━━

...

167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8646 - loss: 0.3402
Epoch 6/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8634 - loss: 0.3328
Epoch 11/100
143/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8659 - loss: 0.3200Epoch 1/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8729 - loss: 0.3362
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8384 - loss: 0.3919  
Epoch 6/100
Epoch 3/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8655 - loss: 0.3202
 67/167 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - accuracy: 0.8546 - loss: 0.3370Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8804 - loss: 0.2729  
Epoch 97/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - accuracy: 0.8606 - loss: 0.3345
Epoch 15/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8587 - loss: 0.3352  
Epoch 7/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8508 - loss: 0.3587 
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0

/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step - accuracy: 0.8701 - loss: 0.3115
Epoch 17/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8649 - loss: 0.3189  
Epoch 17/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8691 - loss: 0.3167  
Epoch 9/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8598 - loss: 0.3381
Epoch 21/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8644 - loss: 0.3209
  1/167 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.8125 - loss: 0.3358Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8685 - loss: 0.3313
Epoch 5/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - accuracy: 0.8661 - loss: 0.3178
Epoch 12/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - accuracy: 0.8800 - loss: 0.2936
Epoch 18/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 940us/step - accuracy: 0.8792 - loss: 0.3044
Epoch 18/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.8733 - loss: 0.3191
Epoch 13/100
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 1

/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/opt/anaconda3/envs/annclass_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.7429 - loss: 0.5259
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step - accuracy: 0.8220 - loss: 0.4313
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step - accuracy: 0.8293 - loss: 0.4062
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 366us/step - accuracy: 0.8435 - loss: 0.3798
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step - accuracy: 0.8447 - loss: 0.3805
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step - accuracy: 0.8491 - loss: 0.3632
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step - accuracy: 0.8574 - loss: 0.3575
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - accuracy: 0.8511 - loss: 0.3700
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - accuracy: 0.8531 - loss: 0.3541
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - accuracy: 0.8616 - loss: 0.3448
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 352us/step - accuracy: 0.8600 - loss: 0.3406
Epoch 12/100
250/250 ━━━━━━━━━━